In [2]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
mode = MPI.MODE_RDONLY
fh = MPI.File.Open(comm, "data-small.txt", mode)

# set read size parameter
file_size = fh.Get_size()
num_process = comm.Get_size()
rank = comm.Get_rank()
chunk_size = file_size / num_process

# read all
buf = bytearray(file_size)
fh.Read_all(buf)

raw = buf.decode()

In [76]:
ts = [(x[0], float(x[1]), int(x[2])) for row in raw.split("\r\n") for x in [row.split(",")] if len(x) == 3]
print(len(ts))
print(ts[:5])
print(ts[-5:])

100000
[('20140804:10:00:13.281486', 782.83, 443355), ('20140804:10:00:13.400113', 955.71, 348603), ('20140804:10:00:13.555512', 1206.38, 467175), ('20140804:10:00:13.435677', 1033.5, 230056), ('20140804:10:00:13.721646', 1107.49, 677398)]
[('20140804:10:02:12.638035', 1421.72, 656296), ('20140804:10:02:12.711615', 1169.65, 92178), ('20140804:10:02:12.906953', 1082.16, 643781), ('20140804:10:02:12.012361', 1109.63, 659206), ('20140804:10:02:12.330983', 975.89, 310894)]


In [94]:
import numpy as np

In [133]:
# find out the longest misplacement

tmp = [x[0] for x in ts]
rank = np.argsort(tmp)
rev = np.maximum(rank - np.arange(len(tmp)), 0)
cdf = np.sort(rev)

print(cdf[99000]) # 99% quantile is 871
print(cdf[99900]) # 99.9% quantile is 1085
print(cdf[-1]) # 100% quantile is 99882
print(np.sum(cdf>0)/len(cdf)) # 50% of records are misplaced

871
1085
99882
0.49394


In [136]:
# duplicates - record, time, (price, vol)
# a paper about filtering tick data
# erratic price on thin volume
# t2 from (t1, t2, t3) is out of wreck. use brownian bridge?